### eval testing

## import lib & var setting

In [ ]:
# Import Pytorch
import torch
from torch import nn

# Import torchvision
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

# Import matplotlib for visulization
import matplotlib.pyplot as plt

import sys 

import cv2

import pandas as pd  # 資料分析套件
import numpy as np
import os
from time import time
import re

In [ ]:
# ===== var setting   ========

# csv_file = '../data/img_feature.csv'
# csv_file = '../data/img_feature_no_masked.csv'
csv_file = '../data/img_conv2.csv'
# random_seed = 42
# dataset_path = "../HAM10000/base_img_data/"
dataset_path = "../HAM10000/images/"
groundtruth_file = '../HAM10000/GroundTruth.csv'

## prepocessing train & test data

### create name to label dict

In [ ]:
# 读取 CSV 文件
# groundtruth_data = pd.read_csv(groundtruth_file)


In [ ]:
# groundtruth_data.head()
# import pandas as pd

# # 假設 df 是您的 DataFrame
# # df = pd.DataFrame(...)

# def find_indices_of_ones(row):
#     # 尋找前六個元素中 1 的位置
#     return [(i-1) for i, x in enumerate(row[:8]) if x == 1]

# # 將函數應用於每行並創建新列 'label'
# groundtruth_data['label'] = groundtruth_data.apply(find_indices_of_ones, axis=1)

# # groundtruth_data.head()

# # find ele == 1
# filename_to_label_dict = groundtruth_data.set_index('image')['label'].to_dict()

# # {'1234' : [2] ,'1235' : [3] } -> {'1234' : 2 ,'1235' : 3 }
# filename_to_label_dict =  {key: value[0] if value else None for key, value in filename_to_label_dict.items()}

# type(filename_to_label_dict['ISIC_0024306'])

# # {'1234' : 2 ,'1235' : 3 } -> {'1234.jpg' : 2 ,'1235.jpg' : 3 }
# filename_to_label_dict =  {key + ".jpg": value for key, value in filename_to_label_dict.items()}




# # check if value is out of range
# null_keys = [key for key, value in filename_to_label_dict.items() if value is None]


# all_values_in_range = all(0 <= value <= 6 for value in filename_to_label_dict.values())

# # 輸出結果
# print("所有的值都在範圍內：" if all_values_in_range else "有些值不在範圍內。")

### 分出train 跟test的data loader

In [ ]:
import random
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from helperFunction.CustomImageDataset import CustomImageDataset 

# # 假設您的圖片和標籤字典如下
# img_dir = dataset_path

# # list all filename from file
# all_files = [f for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))]

# # list all filename from dict
# # all_files = list(filename_to_label_dict.keys())

# random.seed(42)

# num_total_samples = len(all_files)
# split_ratio = 0.8  # 80% 的数据用于训练，20% 用于测试

# # 随机打乱数据集
# random.shuffle(all_files)

# # 计算分割点
# split_idx = int(num_total_samples * split_ratio)

# # 分割数据集
# train_files = all_files[:split_idx]
# test_files = all_files[split_idx:]

gray_mean = sum([0.485, 0.456, 0.406]) / 3
gray_std = sum([0.229, 0.224, 0.225]) / 3


# 可以加入轉換
transform64 = torchvision.transforms.Compose([
    transforms.Resize(size=(64,64)),
    torchvision.transforms.ToTensor()
    # 其他轉換
])

transform_std = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_gray_single_channel = transforms.Compose([
    transforms.Resize(256),  # 调整图像大小至256x256
    transforms.CenterCrop(224),  # 中心裁剪至224x224
    transforms.Grayscale(),  # 将图像转换为灰度图
    transforms.ToTensor(),  # 转换为Tensor
    # 归一化，均值和标准差应根据灰度图的实际分布进行调整
    # 这里使用ImageNet预训练模型的均值和标准差的平均值作为示例
    transforms.Normalize(mean=[0.449], std=[0.226]),
])

transform_gray_pretrain_fit = transforms.Compose([
    # transforms.Grayscale(num_output_channels=1),  # 将图像转换为单通道灰度图像
    # transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # 复制灰度通道到三个通道
    # 其他需要的转换，如 Resize, Normalize 等
    transforms.Resize(256),  # 例如，调整图像大小
    transforms.CenterCrop(224),  # 中心裁剪
    transforms.ToTensor(),  # 转换为Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 归一化
])

transform_gray_CT_img_fit = transforms.Compose([
    # transforms.Grayscale(num_output_channels=1),  # 将图像转换为单通道灰度图像
    # transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # 复制灰度通道到三个通道
    # 其他需要的转换，如 Resize, Normalize 等
    transforms.Resize(256),  # 例如，调整图像大小
    transforms.CenterCrop(224),  # 中心裁剪
    transforms.ToTensor(),  # 转换为Tensor
    transforms.Normalize(mean=[gray_mean,gray_mean,gray_mean], std=[gray_std,gray_std,gray_std]),  # 归一化
])





# transform224_gray_32bit =  transforms.Compose([
#     transforms.Lambda(lambda img: np.array(img).astype(np.float32) / 255.0),  # 将PIL Image转换为NumPy数组，并更改数据类型
#     transforms.Lambda(lambda x: torch.from_numpy(x)),  # 将NumPy数组转换为Tensor
#     transforms.Resize(256),  # 调整图像大小
#     transforms.CenterCrop(224),  # 中心裁剪到224x224
#     # transforms.Lambda(lambda x: x.unsqueeze(0)),  # 增加一个通道维度
#     transforms.Normalize(mean=[0.485], std=[0.229]),  # 使用单个值的均值和标准差进行归一化
# ])



# # 创建训练集和测试集的数据加载器
# train_dataset_64 = CustomImageDataset(img_dir=img_dir, file_to_label_dict={file: filename_to_label_dict[file] for file in train_files}, transform=transform64)
# test_dataset_64 = CustomImageDataset(img_dir=img_dir, file_to_label_dict={file: filename_to_label_dict[file] for file in test_files}, transform=transform64)

# # 创建数据加载器
# train_data_loader_64 = DataLoader(train_dataset_64, batch_size=32, shuffle=True)
# test_data_loader_64 = DataLoader(test_dataset_64, batch_size=32, shuffle=False)


# # 创建训练集和测试集的数据加载器
# train_dataset_224 = CustomImageDataset(img_dir=img_dir, file_to_label_dict={file: filename_to_label_dict[file] for file in train_files}, transform=transform224)
# test_dataset_224 = CustomImageDataset(img_dir=img_dir, file_to_label_dict={file: filename_to_label_dict[file] for file in test_files}, transform=transform224)

# # 创建数据加载器
# train_data_loader_224 = DataLoader(train_dataset_224, batch_size=32, shuffle=True)
# test_data_loader_224 = DataLoader(test_dataset_224, batch_size=32, shuffle=False)

# # 創建 Dataset 和 DataLoader
# dataset = CustomImageDataset(img_dir=img_dir, file_to_label_dict=filename_to_label_dict, transform=transform)
# data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# # 使用 DataLoader
# for images, labels in data_loader:
#     # 這裡進行訓練過程
#     pass

In [ ]:
# @transform test unit

# image = Image.open('../data/chestCTData/images/train/adenocarcinoma/000000 (6).png')

# # 打印图像的格式
# print("Format:", image.format)

# # 打印图像的尺寸
# print("Size:", image.size)

# # 打印图像的模式
# print("Mode:", image.mode)

# # 打印图像的额外信息（如果有的话）
# print("Info:", image.info)

# # 将图像转换为NumPy数组
# image_array = np.array(image)

# # 打印图像数组的形状和部分像素值
# print("Image shape:", image_array.shape)
# print("Some pixel values:", image_array[100:110, 100:110])  # 打印左上角5x5区域的像素值

# transformed_img = transform_gray(image)

# print(transformed_img.shape)

In [ ]:
sys.path.append(os.path.abspath('..'))

from data.chestCTData.chestCTDataloader import ChestCTDataProcessor
from data.HAM10000.ham10000Dataloader import HAM10000DataProcessor

dataContainer = ChestCTDataProcessor(transform=transform_std)
# dataContainer = HAM10000DataProcessor(transform=transform_std)

model_folder_path = '../model/CT chest'
# model_folder_path = '../model/HAM10000'


train_dataloader , test_dataloader = dataContainer.returnDataloaders()


# dataContainer.dataPreprocess()

Classes: ['adenocarcinoma', 'large.cell.carcinoma', 'normal', 'squamous.cell.carcinoma']
Class to index: {'adenocarcinoma': 0, 'large.cell.carcinoma': 1, 'normal': 2, 'squamous.cell.carcinoma': 3}
Samples: [('c:\\Users\\E\\Desktop\\skin_cancer_dataset\\data\\chestCTData\\images\\test\\adenocarcinoma\\000108 (3).png', 0), ('c:\\Users\\E\\Desktop\\skin_cancer_dataset\\data\\chestCTData\\images\\test\\adenocarcinoma\\000109 (2).png', 0), ('c:\\Users\\E\\Desktop\\skin_cancer_dataset\\data\\chestCTData\\images\\test\\adenocarcinoma\\000109 (4).png', 0), ('c:\\Users\\E\\Desktop\\skin_cancer_dataset\\data\\chestCTData\\images\\test\\adenocarcinoma\\000109 (5).png', 0), ('c:\\Users\\E\\Desktop\\skin_cancer_dataset\\data\\chestCTData\\images\\test\\adenocarcinoma\\000112 (2).png', 0)]
First image size: torch.Size([3, 224, 224])
First image label: 0


In [ ]:
img, label = next(iter(train_dataloader))

# Batch size will now be 1, you can cahnge the batch size if you like
print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}") # equal to batch siz

Image shape: torch.Size([32, 3, 224, 224]) -> [batch_size, color_channels, height, width]
Label shape: torch.Size([32])


In [ ]:
# 儲存file list的資訊給XGB訓練時使用
# import json
# with open('test_files_list.json', 'w') as f:
#     json.dump(test_files, f)
# with open('train_files_list.json', 'w') as f:
#     json.dump(train_files, f)

## Bulid TinyVGG model

In [ ]:
# from custom_model.TinyVGG import TinyVGG

# model_0 = TinyVGG(input_shape=3,
#                   hidden_units=10,
#                   output_shape=7)
# model_0

In [ ]:
# forward testing

# print(model_0(img))

In [ ]:
# from torchinfo import summary
# summary(model_0, input_size=[1,3,64,64])


In [ ]:
import torch
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Build VGG16 model

In [ ]:
# import torch
# import torch.nn as nn

# class VGG16(nn.Module):
#     def __init__(self, num_classes=1000):
#         super(VGG16, self).__init__()
        
#         # 定义 VGG16 的卷积层部分
#         self.features = nn.Sequential(
#             nn.Conv2d(3, 64, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(64, 64, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
            
#             nn.Conv2d(64, 128, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(128, 128, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
            
#             nn.Conv2d(128, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(256, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(256, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
            
#             nn.Conv2d(256, 512, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(512, 512, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(512, 512, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
            
#             nn.Conv2d(512, 512, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(512, 512, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(512, 512, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#         )

#         # 定义 VGG16 的全连接层部分
#         self.classifier = nn.Sequential(
#             nn.Linear(512 * 7 * 7, 4096),
#             nn.ReLU(inplace=True),
#             nn.Dropout(),
#             nn.Linear(4096, 4096),
#             nn.ReLU(inplace=True),
#             nn.Dropout(),
#             nn.Linear(4096, num_classes),
#         )

#     def forward(self, x):
#         # 通过卷积层
#         x = self.features(x)
        
#         # 展平操作
#         x = x.view(x.size(0), -1)
        
#         # 通过全连接层
#         x = self.classifier(x)
        
#         return x

# model_vgg16 = VGG16(num_classes=7)

In [ ]:
# from torchinfo import summary
# summary(model_vgg16, input_size=[1,3,224,224])


## Train & test function defination

In [ ]:
# Cteate train_step()
def train_step(model: torch.nn.Module,
        dataloader: torch.utils.data.DataLoader,
        loss_fn: torch.nn.Module,
        optimizer: torch.optim.Optimizer,
        device=device):
  # Put the model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop thourgh data loader data batches
  for batch,(X, y) in enumerate(dataloader):
    # Send data to the target device
    X, y = X.to(device) , y.to(device)

    # 1. Forward pass
    y_pred = model(X)

    # 2. Calculate the loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5 Optimizer step
    optimizer.step()

    # Calculat accuracy metric
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class==y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

In [ ]:
from sklearn.metrics import f1_score
# Create a test step
def test_step(model: torch.nn.Module,
        dataloader: torch.utils.data.DataLoader,
        loss_fn: torch.nn.Module,
        device=device):
  # Put the model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc, all_preds, all_labels = 0, 0, [], []

  with torch.inference_mode():
    # Loop thourgh data loader data batches
    for batch,(X, y) in enumerate(dataloader):
      # Send data to the target device
      X, y = X.to(device) , y.to(device)

      # 1. Forward pass
      test_pred_logits = model(X)

      # 2. Calculate the loss
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()


      # Calculate accuracy
      test_pred_labels = test_pred_logits.argmax(dim=1) # Maybe same as softmax results?
      test_acc += (test_pred_labels==y).sum().item() / len(test_pred_labels)

      all_preds.extend(test_pred_labels.cpu().numpy())
      all_labels.extend(y.cpu().numpy())

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  # cal F1 score
  f1 = f1_score(all_labels, all_preds, average='weighted')

  return test_loss, test_acc, f1

In [ ]:
from tqdm.auto import tqdm

# 1. Create a train function that takes in various model parameters + optimizer + dataloader + loss functoin
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5,
          best_model_name: str = "best_model.pth",
          device=device,
          enable_early_stop=True):
  # 2. Create empty results dicitonary
  results = {"train_loss": [],
        "train_acc":[],
        "test_loss": [],
        "test_acc":[]}
  
  # init best model loss var
  best_test_loss = float('inf')
  best_model = None
  early_stop_count = 0
  early_stop_limit = 7

  # 3. Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                        dataloader=train_dataloader,
                        loss_fn=loss_fn,
                        optimizer=optimizer,
                        device=device)
    test_loss, test_acc, test_f1 = test_step(model=model,
                      dataloader=test_dataloader,
                      loss_fn=loss_fn,
                      device=device)
    # 4. Print out what's happening
    print(f"Epoch: {epoch} | Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f} | Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f} |  F1 score: {test_f1:.4f}   ")

    # 5. Update results dictionary
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

    if test_loss < best_test_loss:
      best_test_loss = test_loss
      best_test_acc = test_acc
      best_test_f1 = test_f1
      # best_model = model.state_dict()
      torch.save(model.state_dict(), best_model_name)
      early_stop_count = 0
    else:
      if enable_early_stop:
        early_stop_count += 1
    
    if early_stop_count >= early_stop_limit:
      print("Early stopping triggered.")
      break

  # 6. Return the filled results at the end of the eopchs
      
  # save best model
  # torch.save(best_model, best_model_name)
  print("the best test acc is :{:.4f}".format(best_test_acc))
  print("the best F1 score is :{:.4f}".format(best_test_f1))
  return results

c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from helperFunction.TrainHelper import TrainingHelper

num_CT_classes = 4
num_skin_classes = 7

num_classes = num_skin_classes

# Train model

## VGG

load preTrained VGG16, num_class = 1000

In [ ]:
import torchvision.models as models

model_preTrained_VGG16 = models.vgg16(pretrained=True).to(device)     # 使用內建的 model 

optimizer = torch.optim.SGD(model_preTrained_VGG16.parameters(), lr = 0.001) # 選擇你想用的 optimizer
# optimizer = torch.optim.Adam(model_VGG16.parameters(), lr =0.01)

# Loss function
loss_fn = nn.CrossEntropyLoss()                # 選擇想用的 loss functionmodel_vgg16 = 

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_VGG_results = train(model=model_preTrained_VGG16,
              train_dataloader=train_dataloader,
              test_dataloader=test_dataloader,
              optimizer=optimizer,
              loss_fn=loss_fn,
              best_model_name= model_folder_path + "/best_model_pretrain_VGG16.pth",
              epochs=40)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|          | 0/40 [00:00<?, ?it/s]

Epoch: 0 | Train loss: 3.1632 | Train acc: 0.3028 | Test loss: 1.3673 | Test acc: 0.4921 |  F1 score: 0.4772   


  2%|▎         | 1/40 [00:16<10:43, 16.51s/it]

Epoch: 1 | Train loss: 1.2012 | Train acc: 0.5963 | Test loss: 1.0539 | Test acc: 0.5698 |  F1 score: 0.5563   


  5%|▌         | 2/40 [00:28<08:53, 14.03s/it]

...

Epoch: 15 | Train loss: 0.1198 | Train acc: 0.9719 | Test loss: 0.7674 | Test acc: 0.6954 |  F1 score: 0.6817   


 42%|████▎     | 17/40 [03:22<04:18, 11.23s/it]

Epoch: 16 | Train loss: 0.1373 | Train acc: 0.9609 | Test loss: 0.8017 | Test acc: 0.6829 |  F1 score: 0.6634   


 42%|████▎     | 17/40 [03:29<04:44, 12.35s/it]


KeyboardInterrupt: 

output == 7 pretrain VGG

In [ ]:
import torchvision.models as models

model_preTrained_VGG16 = models.vgg16(pretrained=True)    # 使用內建的 model 


optimizer = torch.optim.SGD(model_preTrained_VGG16.parameters(), lr = 0.001) # 選擇你想用的 optimizer

# optimizer = torch.optim.Adam(model_VGG16.parameters(), lr =0.01)

# Loss function
loss_fn = nn.CrossEntropyLoss()                # 選擇想用的 loss functionmodel_vgg16 = 

#=========
import torch
import torch.nn as nn
import torchvision.models as models




# 获取分类器中最后一个全连接层之前的部分
classifier = list(model_preTrained_VGG16.classifier.children())[:-1]

# 移除原始模型的最后一个全连接层
# 并添加一个新的全连接层，输出特征数为 7
classifier.append(torch.nn.Linear(4096, 7))

# 替换原始模型的分类器
model_preTrained_VGG16.classifier = torch.nn.Sequential(*classifier)

# ============


model_preTrained_VGG16 =  model_preTrained_VGG16.to(device)

from torchinfo import summary

summary(model_preTrained_VGG16, input_size=[1,3,224,224])



# Start the timer


# from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_VGG16_results = train(model=model_preTrained_VGG16,
              train_dataloader=train_dataloader,
              test_dataloader=test_dataloader,
              optimizer=optimizer,
              loss_fn=loss_fn,
              best_model_name=model_folder_path + "\\best_model_pretrain_VGG16.pth",
              epochs=40)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|          | 0/40 [00:00<?, ?it/s]

Epoch: 0 | Train loss: 2.0036 | Train acc: 0.1453 | Test loss: 1.7867 | Test acc: 0.2851 |  F1 score: 0.2716   


  2%|▎         | 1/40 [00:12<07:59, 12.31s/it]

Epoch: 1 | Train loss: 1.6366 | Train acc: 0.3247 | Test loss: 1.5132 | Test acc: 0.4745 |  F1 score: 0.4503   


  5%|▌         | 2/40 [00:24<07:44, 12.23s/it]

...

 88%|████████▊ | 35/40 [06:43<00:55, 11.10s/it]

Epoch: 34 | Train loss: 0.1237 | Train acc: 0.9531 | Test loss: 0.5943 | Test acc: 0.7190 |  F1 score: 0.7174   
Epoch: 35 | Train loss: 0.1048 | Train acc: 0.9734 | Test loss: 0.5770 | Test acc: 0.7397 |  F1 score: 0.7391   


 92%|█████████▎| 37/40 [07:05<00:33, 11.16s/it]

Epoch: 36 | Train loss: 0.1102 | Train acc: 0.9641 | Test loss: 0.5821 | Test acc: 0.7622 |  F1 score: 0.7597   
Epoch: 37 | Train loss: 0.0860 | Train acc: 0.9750 | Test loss: 0.5656 | Test acc: 0.7477 |  F1 score: 0.7470   


 95%|█████████▌| 38/40 [07:17<00:22, 11.37s/it]

output == 7 non-pretrain VGG

In [ ]:
import torchvision.models as models

model_VGG16_7 = models.vgg16(pretrained=False)    # 使用內建的 model 


optimizer = torch.optim.SGD(model_VGG16_7.parameters(), lr = 0.001) # 選擇你想用的 optimizer

# optimizer = torch.optim.Adam(model_VGG16.parameters(), lr =0.01)

# Loss function
loss_fn = nn.CrossEntropyLoss()                # 選擇想用的 loss functionmodel_vgg16 = 

#=========
import torch
import torch.nn as nn
import torchvision.models as models




# 获取分类器中最后一个全连接层之前的部分
classifier = list(model_VGG16_7.classifier.children())[:-1]

# 移除原始模型的最后一个全连接层
# 并添加一个新的全连接层，输出特征数为 7
classifier.append(torch.nn.Linear(4096, 7))

# 替换原始模型的分类器
model_VGG16_7.classifier = torch.nn.Sequential(*classifier)

# ============


model_VGG16_7 =  model_VGG16_7.to(device)



# from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_VGG16_7_results = train(model=model_preTrained_VGG16_7,
              train_dataloader=train_data_loader_224,
              test_dataloader=test_data_loader_224,
              optimizer=optimizer,
              loss_fn=loss_fn,
              best_model_name="best_model_VGG16_7.pth",
              epochs=120)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")


c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
  1%|          | 1/120 [01:38<3:15:23, 98.52s/it]

Epoch: 0 | Train loss: 0.3846 | Train acc: 0.8590 | Test loss: 0.6782 | Test acc: 0.7466 |  F1 score: 0.7649   


  2%|▏         | 2/120 [03:16<3:13:32, 98.42s/it]

Epoch: 1 | Train loss: 0.3843 | Train acc: 0.8542 | Test loss: 0.6782 | Test acc: 0.7466 |  F1 score: 0.7649   


  2%|▎         | 3/120 [04:55<3:12:00, 98.46s/it]

...

 99%|█████████▉| 119/120 [3:16:54<01:38, 98.56s/it]

Epoch: 118 | Train loss: 0.3831 | Train acc: 0.8608 | Test loss: 0.6782 | Test acc: 0.7466 |  F1 score: 0.7649   


100%|██████████| 120/120 [3:18:33<00:00, 99.28s/it]

Epoch: 119 | Train loss: 0.3847 | Train acc: 0.8533 | Test loss: 0.6782 | Test acc: 0.7466 |  F1 score: 0.7649   


the best test acc is :0.7466
Total training time: 11914.561 seconds


## Resnet 18

pretrain resnet

In [ ]:
import torchvision.models as models

model_preTrained_Resnet18 = models.resnet18(pretrained=True).to(device)     # 使用內建的 model 

summary(model_preTrained_Resnet18, input_size=[1,3,224,224])

optimizer = torch.optim.SGD(model_preTrained_Resnet18.parameters(), lr = 0.001) # 選擇你想用的 optimizer

# optimizer = torch.optim.Adam(model_Resnet18.parameters(), lr =0.01)

# Loss function
loss_fn = nn.CrossEntropyLoss()                # 選擇想用的 loss functionmodel_vgg16 = 




# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_VGG_results = train(model=model_preTrained_Resnet18,
              train_dataloader=train_data_loader_224,
              test_dataloader=test_data_loader_224,
              optimizer=optimizer,
              loss_fn=loss_fn,
              best_model_name="best_model_pretrain_Resnet18.pth",
              epochs=40)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  2%|▎         | 1/40 [01:02<40:23, 62.14s/it]

Epoch: 0 | Train loss: 2.4491 | Train acc: 0.5971 | Test loss: 1.2487 | Test acc: 0.6910 |  F1 score: 0.6630   


  5%|▌         | 2/40 [02:04<39:37, 62.56s/it]

Epoch: 1 | Train loss: 0.8927 | Train acc: 0.7424 | Test loss: 0.8683 | Test acc: 0.7302 |  F1 score: 0.7014   


  8%|▊         | 3/40 [03:08<38:43, 62.79s/it]

...

Epoch: 37 | Train loss: 0.0252 | Train acc: 0.9991 | Test loss: 0.7103 | Test acc: 0.7879 |  F1 score: 0.7794   


 98%|█████████▊| 39/40 [41:14<01:06, 66.34s/it]

Epoch: 38 | Train loss: 0.0230 | Train acc: 0.9986 | Test loss: 0.7232 | Test acc: 0.7899 |  F1 score: 0.7801   


100%|██████████| 40/40 [42:20<00:00, 63.52s/it]

Epoch: 39 | Train loss: 0.0211 | Train acc: 0.9984 | Test loss: 0.7264 | Test acc: 0.7878 |  F1 score: 0.7791   
the best test acc is :0.7815
Total training time: 2540.734 seconds


output == 7 pretrain resnet

In [ ]:
import torchvision.models as models

model_preTrained_Resnet18_7 = models.resnet18(pretrained=True)     # 使用內建的 model 


# optimizer = torch.optim.SGD(model_preTrained_Resnet18_7.parameters(), lr = 0.001) # 選擇你想用的 optimizer
optimizer = torch.optim.SGD(model_preTrained_Resnet18_7.parameters(), lr = 0.001, momentum=0.9) # 選擇你想用的 optimizer

# optimizer = torch.optim.Adam(model_Resnet18_7.parameters(), lr =0.01)

# Loss function
loss_fn = nn.CrossEntropyLoss()                # 選擇想用的 loss functionmodel_vgg16 = 

# ========================

# change fully connected layer to 7
num_ftrs = model_preTrained_Resnet18_7.fc.in_features
model_preTrained_Resnet18_7.fc = nn.Linear(num_ftrs, 7)

# device setting
model_preTrained_Resnet18_7 = model_preTrained_Resnet18_7.to(device)

# ===========================

# summary(model_preTrained_Resnet18_7)


# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_VGG_results = train(model=model_preTrained_Resnet18_7,
              train_dataloader=train_data_loader_224,
              test_dataloader=test_data_loader_224,
              optimizer=optimizer,
              loss_fn=loss_fn,
              best_model_name="best_model_pretrain_Resnet18_7.pth",
              epochs=40)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/40 [00:29<?, ?it/s]


KeyboardInterrupt: 

output == 7 non-pretrain resnet

In [ ]:
import torchvision.models as models

model_preTrained_Resnet18 = models.resnet18(pretrained=False).to(device)     # 使用內建的 model 

summary(model_preTrained_Resnet18, input_size=[1,3,224,224])

optimizer = torch.optim.SGD(model_preTrained_Resnet18.parameters(), lr = 0.001) # 選擇你想用的 optimizer

# optimizer = torch.optim.Adam(model_Resnet18.parameters(), lr =0.01)

# Loss function
loss_fn = nn.CrossEntropyLoss()                # 選擇想用的 loss functionmodel_vgg16 = 




# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_VGG_results = train(model=model_preTrained_Resnet18,
              train_dataloader=train_data_loader_224,
              test_dataloader=test_data_loader_224,
              optimizer=optimizer,
              loss_fn=loss_fn,
              best_model_name="best_model_Resnet18.pth",
              epochs=120)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
  1%|          | 1/120 [01:02<2:03:41, 62.37s/it]

Epoch: 0 | Train loss: 2.5158 | Train acc: 0.6380 | Test loss: 1.5130 | Test acc: 0.6826 |  F1 score: 0.5538   


  2%|▏         | 2/120 [02:05<2:03:11, 62.64s/it]

Epoch: 1 | Train loss: 1.2750 | Train acc: 0.6729 | Test loss: 1.1878 | Test acc: 0.6876 |  F1 score: 0.5695   


  2%|▎         | 3/120 [03:07<2:02:09, 62.65s/it]

...

Epoch: 117 | Train loss: 0.0678 | Train acc: 0.9853 | Test loss: 1.1070 | Test acc: 0.7281 |  F1 score: 0.7185   


 99%|█████████▉| 119/120 [2:03:53<01:02, 62.51s/it]

Epoch: 118 | Train loss: 0.0760 | Train acc: 0.9819 | Test loss: 1.1065 | Test acc: 0.7446 |  F1 score: 0.7119   


100%|██████████| 120/120 [2:04:55<00:00, 62.46s/it]

Epoch: 119 | Train loss: 0.0631 | Train acc: 0.9860 | Test loss: 1.5772 | Test acc: 0.7105 |  F1 score: 0.6729   
the best test acc is :0.7354
Total training time: 7495.337 seconds


output == 7 pretrain resnet

In [ ]:
import torchvision.models as models

model_preTrained_Resnet18_7 = models.resnet18(pretrained=True)     # 使用內建的 model 


# optimizer = torch.optim.SGD(model_preTrained_Resnet18_7.parameters(), lr = 0.001) # 選擇你想用的 optimizer
optimizer = torch.optim.SGD(model_preTrained_Resnet18_7.parameters(), lr = 0.001, momentum=0.9) # 選擇你想用的 optimizer

# optimizer = torch.optim.Adam(model_Resnet18_7.parameters(), lr =0.01)

# Loss function
loss_fn = nn.CrossEntropyLoss()                # 選擇想用的 loss functionmodel_vgg16 = 

# ========================

# change fully connected layer to 7
num_ftrs = model_preTrained_Resnet18_7.fc.in_features
model_preTrained_Resnet18_7.fc = nn.Linear(num_ftrs, 7)

# device setting
model_preTrained_Resnet18_7 = model_preTrained_Resnet18_7.to(device)

# ===========================

summary(model_preTrained_Resnet18_7)


# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_VGG_results = train(model=model_preTrained_Resnet18_7,
              train_dataloader=train_data_loader_224,
              test_dataloader=test_data_loader_224,
              optimizer=optimizer,
              loss_fn=loss_fn,
              best_model_name="best_model_Resnet18_7.pth",
              epochs=120)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  1%|          | 1/120 [01:02<2:03:53, 62.47s/it]

Epoch: 0 | Train loss: 1.4069 | Train acc: 0.5632 | Test loss: 0.9721 | Test acc: 0.7389 |  F1 score: 0.7194   


  2%|▏         | 2/120 [02:04<2:02:39, 62.37s/it]

Epoch: 1 | Train loss: 0.7745 | Train acc: 0.7795 | Test loss: 0.7548 | Test acc: 0.7657 |  F1 score: 0.7325   


  2%|▎         | 3/120 [03:07<2:02:14, 62.68s/it]

...

Epoch: 117 | Train loss: 0.0042 | Train acc: 0.9998 | Test loss: 0.6730 | Test acc: 0.8404 |  F1 score: 0.8301   


 99%|█████████▉| 119/120 [2:04:14<01:02, 62.98s/it]

Epoch: 118 | Train loss: 0.0039 | Train acc: 1.0000 | Test loss: 0.6627 | Test acc: 0.8418 |  F1 score: 0.8317   


100%|██████████| 120/120 [2:05:17<00:00, 62.64s/it]

Epoch: 119 | Train loss: 0.0037 | Train acc: 1.0000 | Test loss: 0.6664 | Test acc: 0.8475 |  F1 score: 0.8382   
the best test acc is :0.8179
Total training time: 7517.327 seconds


In [ ]:
import torchvision.models as models

model_preTrained_Resnet50 = models.resnet50(pretrained=True)     # 使用內建的 model 


# optimizer = torch.optim.SGD(model_preTrained_Resnet50.parameters(), lr = 0.001) # 選擇你想用的 optimizer
optimizer = torch.optim.SGD(model_preTrained_Resnet50.parameters(), lr = 0.001, momentum=0.9) # 選擇你想用的 optimizer

# optimizer = torch.optim.Adam(model_Resnet50.parameters(), lr =0.01)

# Loss function
loss_fn = nn.CrossEntropyLoss()                # 選擇想用的 loss functionmodel_vgg16 = 


# change fully connected layer to 7
num_ftrs = model_preTrained_Resnet50.fc.in_features
model_preTrained_Resnet50.fc = nn.Linear(num_ftrs, 7)

# device setting
model_preTrained_Resnet50 = model_preTrained_Resnet50.to(device)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_VGG_results = train(model=model_preTrained_Resnet50,
              train_dataloader=train_data_loader_224,
              test_dataloader=test_data_loader_224,
              optimizer=optimizer,
              loss_fn=loss_fn,
              best_model_name="best_model_pretrain_Resnet50_7.pth",
              epochs=40)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  2%|▎         | 1/40 [01:24<55:11, 84.91s/it]

Epoch: 0 | Train loss: 1.3020 | Train acc: 0.6268 | Test loss: 0.9094 | Test acc: 0.7448 |  F1 score: 0.7212   


  5%|▌         | 2/40 [02:50<53:50, 85.02s/it]

Epoch: 1 | Train loss: 0.7284 | Train acc: 0.7907 | Test loss: 0.7710 | Test acc: 0.7575 |  F1 score: 0.7368   


  8%|▊         | 3/40 [04:14<52:24, 85.00s/it]

...

Epoch: 37 | Train loss: 0.0160 | Train acc: 0.9968 | Test loss: 0.6162 | Test acc: 0.8428 |  F1 score: 0.8387   


 98%|█████████▊| 39/40 [55:19<01:25, 85.23s/it]

Epoch: 38 | Train loss: 0.0212 | Train acc: 0.9957 | Test loss: 0.5908 | Test acc: 0.8523 |  F1 score: 0.8461   


100%|██████████| 40/40 [56:44<00:00, 85.12s/it]

Epoch: 39 | Train loss: 0.0086 | Train acc: 0.9993 | Test loss: 0.5775 | Test acc: 0.8559 |  F1 score: 0.8506   
the best test acc is :0.8309
Total training time: 3404.949 seconds


# model output

In [ ]:
import torchvision.models as models

new_load_model = models.resnet18(pretrained=True)     # 使用內建的 model 

optimizer = torch.optim.SGD(new_load_model.parameters(), lr = 0.001) # 選擇你想用的 optimizer
# optimizer = torch.optim.Adam(model_VGG16.parameters(), lr =0.01)

# Loss function
loss_fn = nn.CrossEntropyLoss()                # 選擇想用的 loss functionmodel_vgg16 = 




# VGG16 setting
# =============
# # 获取分类器中最后一个全连接层之前的部分
# classifier = list(new_load_VGG16.classifier.children())[:-1]

# # 移除原始模型的最后一个全连接层
# # 并添加一个新的全连接层，输出特征数为 7
# classifier.append(torch.nn.Linear(4096, 7))

# # 替换原始模型的分类器
# new_load_VGG16.classifier = torch.nn.Sequential(*classifier)

# ============

# resnet setting
#  ============
# change fully connected layer to 7
num_ftrs = new_load_model.fc.in_features
new_load_model.fc = nn.Linear(num_ftrs, 7)
#  ============


new_load_model =  new_load_model.to(device)

new_load_model.load_state_dict(torch.load("./best_model_pretrain_Resnet18_7.pth", map_location=device))

from torchinfo import summary

# summary(model_preTrained_VGG16_7, input_size=[1,3,224,224])





# Start the timer
from timeit import default_timer as timer
start_time = timer()

new_load_model.eval()

loss, acc, f1 = test_step(new_load_model,
                        loss_fn=loss_fn,
                        dataloader=test_data_loader_224,
                        device=device)

print("acc:{:.4f}".format(acc))
print("f1:{:.4f}".format(f1))



c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


acc:0.8312
f1:0.8258


In [ ]:
print(loss)
print("acc:{:.4f}".format(acc))
print("f1:{:.4f}".format(f1))


0.6782166653058745
acc:0.7466
f1:0.7649


In [ ]:
# image_path = "./HAM10000/images/ISIC_0024306.jpg"

# def img2tensor(img_path):
#     img = cv2.imread(img_path)
#     img = cv2.resize(img, (224, 224))

#     data = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).float() / 255.0
#     return data

# def flattenTensor(tensor):
#     tensor = torch.squeeze(tensor)
#     flatted_tensor = tensor.view(-1)
#     return flatted_tensor

# # output = conv_layer(input_data)
# # print(input_data.shape)
# # print(output.shape)

# # output = conv_block(input_data)
# # output = conv_block_2(output)
# input_data = img2tensor(image_path)
# input_data = input_data.to(device)

# model_VGG16.eval()
# with torch.no_grad():
#     output = model_VGG16(input_data)
# prob = torch.softmax(output, dim=1)
# print(input_data.shape)
# print(output.shape)
# print(prob)
# print(flattenTensor(output).shape)

In [ ]:
# import os
# import csv
# import time

# input_folder = "HAM10000/images/"
# csvfile_name = "./img_conv3.csv"


# res = []

# # open every img
# start = time.time()
# for filename in os.listdir(input_folder):

#     image_path  = os.path.join(input_folder, filename)
#     # print(image_path)
#     img = img2tensor(image_path)
#     tensor = conv3(img)
#     tensor = flattenTensor(tensor)
#     vec = tensor.tolist()

#     vec = [filename] + vec
#     res.append(vec)
# end = time.time()

# print("time cost : {}".format(end - start))

# # print(res)
# with open(csvfile_name, 'w', newline='', encoding='utf-8') as csvfile:
#     # try:
#     csvwriter = csv.writer(csvfile)
#     csvwriter.writerows(res)
#     # except Exception as e:
#     #     logging.error("csv error : %s" , e)
#     #     print('csv encoding error')

## Tiny VGG

input wiil be [3, 64, 64]

In [ ]:
# # Set random seeds

# torch.manual_seed(42)
# torch.cuda.manual_seed(42)

# # Set number of epochs
# NUM_EPOCHS = 30

# # Recreate an instance of TinyVGG
# model_0 = TinyVGG(input_shape=3, # numbers of color channels
#           hidden_units=10,
#           output_shape=7).to(device)

# # Setup loss function and optimizer
# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(params=model_0.parameters(),
#                 lr =0.001)

# # Start the timer
# from timeit import default_timer as timer
# start_time = timer()

# # Train model_0
# model_0_results = train(model=model_0,
#               train_dataloader=train_data_loader,
#               test_dataloader=test_data_loader,
#               optimizer=optimizer,
#               loss_fn=loss_fn,
#               epochs=NUM_EPOCHS)

# # End the timer and print out how long it took
# end_time = timer()
# print(f"Total training time: {end_time-start_time:.3f} seconds")


 83%|████████▎ | 25/30 [25:30<05:04, 60.83s/it]
Epoch: 24 | Train loss: 0.5824 | Train acc: 0.7830 | Test loss: 0.8261 | Test acc: 0.7118  
 87%|████████▋ | 26/30 [26:31<04:03, 60.86s/it]
Epoch: 25 | Train loss: 0.5774 | Train acc: 0.7839 | Test loss: 0.8404 | Test acc: 0.7162  
 90%|█████████ | 27/30 [27:32<03:02, 60.86s/it]
Epoch: 26 | Train loss: 0.5776 | Train acc: 0.7862 | Test loss: 0.8416 | Test acc: 0.7101  
 93%|█████████▎| 28/30 [28:33<02:01, 60.82s/it]
Epoch: 27 | Train loss: 0.5675 | Train acc: 0.7873 | Test loss: 0.8176 | Test acc: 0.7065  
 97%|█████████▋| 29/30 [29:34<01:00, 60.79s/it]
Epoch: 28 | Train loss: 0.5592 | Train acc: 0.7901 | Test loss: 0.8350 | Test acc: 0.7147  
100%|██████████| 30/30 [30:35<00:00, 61.17s/it]
Epoch: 29 | Train loss: 0.5536 | Train acc: 0.7954 | Test loss: 0.8560 | Test acc: 0.7035  
Total training time: 1835.003 seconds
